In [2]:
from demucs import pretrained
import torch
from demucs.apply import apply_model
import timeit

In [8]:
x = torch.randn(1, 2, 44100 * 30)  # 30 seconds of white noise for the test
models_to_test = ['mdx', 'htdemucs', 'htdemucs_ft', 'htdemucs_6s', 'hdemucs_mmi', 'mdx_extra']

## Env
### Hardware
- 2022 MacBook Pro
- Apple M2 Chip
- 24 GB RAM
### Software
- Python 3.10.11
- torch-2.1.0.dev20230601
-  torchaudio-2.1.0.dev20230601

In [9]:
for model_name in models_to_test:
    model = pretrained.get_model(model_name)
    
    print(f"Testing {model_name}")

    mps_time = timeit.timeit(lambda: apply_model(model, x, device='mps'), number=3)
    cpu_time = timeit.timeit(lambda: apply_model(model, x, device='cpu'), number=3)

    print(f"CPU time: {cpu_time:.2f}s, MPS time: {mps_time:.2f}s")
    print(f"Speedup: {cpu_time / mps_time:.2f}x")
    print()

Testing mdx
CPU time: 67.53s, MPS time: 224.81s
Speedup: 0.30x

Testing htdemucs
CPU time: 52.90s, MPS time: 14.44s
Speedup: 3.66x

Testing htdemucs_ft
CPU time: 210.97s, MPS time: 49.38s
Speedup: 4.27x

Testing htdemucs_6s
CPU time: 50.26s, MPS time: 12.76s
Speedup: 3.94x

Testing hdemucs_mmi
CPU time: 21.59s, MPS time: 62.02s
Speedup: 0.35x

Testing mdx_extra
CPU time: 79.57s, MPS time: 210.56s
Speedup: 0.38x

